In [1]:
# Require Pandas version >= 2.2.2
import pandas as pd
assert pd.__version__ >= '2.2.2', 'Please update Pandas to version 2.2.2 or newer'
import pandas as pd, os, pyreadr
import warnings
import time
import numpy as  np
import pandas as pd
import pandas as pd
import visualization as viz
import data_loading as dl
import risk_pricing as rp
import statsmodels.api as sm
import statsmodels.formula.api as smf
import importlib
importlib.reload(rp)
importlib.reload(viz)
importlib.reload(dl)
warnings.filterwarnings("ignore")
start_time = time.time()
source_folder_path = "/Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/src/"

In [2]:
# # Creating additional models
# import matplotlib.pyplot as plt
# embeddings = Embeddings()
# EmbeddingsHKRModel.plot_cosine_similarity(["intellectual property", "patent", "knowledge capital", "research and development", "key talent"])
# Save last plot:
# plt.savefig("cosine_similarity.png")
# base_path = "/Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/text/"
# dir_path = os.path.join(base_path)
# os.makedirs(dir_path, exist_ok=True)
# figfolder = os.path.join(dir_path, "")
# # Save to the folder:
# plt.savefig(figfolder + "cosine_similarity_terms.png")
# EmbeddingsHKRModel().from_topic_similarity(embeddings, "emb_knocap", "knowledge capital", to_csv = True)
# EmbeddingsHKRModel().from_topic_similarity(embeddings, "emb_patents", "patents", to_csv = True)
# EmbeddingsHKRModel().from_topic_similarity(embeddings, "emb_intcap", "intangible capital", to_csv = True)

In [3]:
import risk_pricing as rp
import importlib
import embedding_tools as et
import visualization as viz
from embedding_tools import Embeddings
from embedding_tools import EmbeddingsHKRModel
importlib.reload(rp)
importlib.reload(viz)
quantiles = 10
# embeddings = Embeddings()
# EmbeddingsHKRModel().from_topic_similarity(embeddings, "emb_kt", "key talent", to_csv = True)
# cases_list = [("embeddings_km10_ipcs", "pfkki3me3mb", 3)]
cases_list = [#("embeddings_km10_ipcs", "pfkk2me3mb", 3),
              #("embeddings_km10_ipcs", "pfkk2me3mb", 5),
              ("embeddings_km10_ipcs", "pfkki3me3mb", 3)#,
            #  ("embeddings_km10_ipcs", "pfkki3me3mb", 5),
            #  ("embeddings_km10_ipcs", "pfkki3me3mb", 10),
            #   ("dicfullmc10thr10defnob40noa0_8_hdp", "pfkk2me3mb", 5),
            #   ("dicfullmc10thr10defnob40noa0_8_4t", "pfkk2me3mb", 5),
              #("embeddings_km10_ipcs", "pfkk2me3mb", 10), # GMM does not converge
            #  ("emb_patents", "pfkki3me3mb", 3)
             ]

for modelname, pfname, quantiles in cases_list:
    figfolder, add_innerkk_pf, cuts = viz.setup(quantiles, modelname, pfname)
    amazon_nov01_short, cequity_mapper, ff3fw, ff5fw, ff3fm, ff5fm, topic_map_unlabeled, comparison_measures, stoxmo_orig, stoxda_orig, stoxwe_orig, compustat_pt = \
        dl.load_dataframes(modelname, start_time, clean_again = False)
    print("Data loaded")
    topic_map = viz.label_topic_map(topic_map_unlabeled, modelname, cuts = cuts)
    fmb_5, fmb_3, fmb_1, df_betas5, df_betas3, df_betas1, eret_we5, eret_we3, stoxwe_add, eret_we_pct5, eret_we_pct3 = rp.famaMacBethMultiCase(pfname, add_innerkk_pf, cuts, cequity_mapper, ff3fw, ff5fw, stoxwe_orig, topic_map, log_returns = False)
    viz.explore_topic_map(topic_map, figfolder)
    # viz.explore_fmb([fmb_1, fmb_3, fmb_5], figfolder)
    # viz.explore_eret_we(eret_we3, eret_we5, eret_we_pct3, eret_we_pct5, figfolder, gmm_case = 5)
    # viz.tex_compare_kk_measures(comparison_measures, figfolder)
    # viz.plot_returns(stoxwe_add, figfolder)
    # viz.amazon_graph(amazon_nov01_short, figfolder)
    # # viz.filecounter(figfolder)
    # viz.explore_stoxda(stoxda_orig, cequity_mapper, topic_map, figfolder)
    # viz.explore_betas(df_betas3, quantiles, figfolder)

Running model embeddings_km10_ipcs with 3 quantiles and pfkki3me3mb portfolio
Data loaded
Executing label_topic_map...
Running Fama-MacBeth regressions: 5 factors


UnboundLocalError: local variable 'stoxwe' referenced before assignment

In [4]:
topic_map_unlabeled.columns

Index(['max_topic', 'topic_kk', 'year', 'CIK', 'CUSIP8', 'LPERMNO', 'sic',
       'cik', 'gvkey', 'conm', 'naics4', 'ind12', 'sic3', 'hi_tech', 'Skill',
       'xi_yeartotal', 'xi_cumsum', 'signed', 'K_int_Know', 'K_int', 'at',
       'prcc_f', 'prcc_c', 'ppegt', 'csho', 'ceq', 'cusip', 'exchg', 'xrd',
       'xir_cumsum', 'xir_total'],
      dtype='object')

In [5]:
# import pandas as pd
# import numpy as np

subset = topic_map_unlabeled.copy()
subset['xrd_type'] = np.where(subset['xrd'].isna(), 'nan',
                              np.where(subset['xrd'] == 0, 'zero', 'positive'))
subset['log_at'] = np.log(subset['at'])
subset['log_ppegt'] = np.log(subset['ppegt'])
subset['log_xrd'] = np.log(subset['xrd'])
subset['xi_yeartotal'] = subset['xi_yeartotal'].astype(float)

columns_to_analyze = ['signed', 'xrd', 'topic_kk', 'Skill', 'at', 'ppegt', 'xi_yeartotal']

def analyze_group(group):
    return group[columns_to_analyze].agg(['mean', 'median', 'std'])

result = pd.concat([
    analyze_group(subset[subset['xrd_type'] == 'nan']),
    analyze_group(subset[subset['xrd_type'] == 'zero']),
    analyze_group(subset[subset['xrd_type'] == 'positive'])
], keys=['nan', 'zero', 'positive'], names=['xrd_type', 'statistic'])
result

import pandas as pd
import numpy as np

subset = topic_map_unlabeled.copy()
subset['log_at'] = np.log(subset['at'])
subset['log_ppegt'] = np.log(subset['ppegt'])
subset['log_xrd'] = np.log(subset['xrd'])
subset['xi_yeartotal'] = subset['xi_yeartotal'].astype(float)

columns_to_analyze = ['xrd', 'topic_kk', 'Skill', 'at', 'ppegt', 'xi_yeartotal']

def analyze_group(group):
    return group[columns_to_analyze].agg(['median', 'mean', 'std'])

result = pd.concat([
    analyze_group(subset[subset['signed'] == 0]),
    analyze_group(subset[subset['signed'] == 1])
], keys=['Unsigned (0)', 'Signed (1)'], names=['signed', 'statistic'])

# round KKR and Skill to 3 decimal places:
result['topic_kk'] = result['topic_kk'].round(3)
result['Skill'] = result['Skill'].round(3)
# round ohter columns to intergeR:
result['at'] = result['at'].astype(int)
result['ppegt'] = result['ppegt'].astype(int)
result['xi_yeartotal'] = result['xi_yeartotal'].astype(int)
result['xrd'] = result['xrd'].astype(int)
# result['log_xrd'] = result['log_xrd'].round(3)
# result['log_at'] = result['log_at'].round(3)
# result['log_ppegt'] = result['log_ppegt'].round(3)

result.columns = ['R\\&D', 'KKR', 'Skill', 'Assets', 'PPE', 'Patent Value']
result
# Convert the result to LaTeX
latex_table = result.to_latex(multicolumn=True, multirow=True, float_format="{:0.3f}".format)

# Save the LaTeX table to a file
with open('signed_analysis_table.tex', 'w') as f:
    f.write(latex_table)

print("LaTeX table has been saved to 'signed_analysis_table.tex'")

LaTeX table has been saved to 'signed_analysis_table.tex'


In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming topic_map_unlabeled is your DataFrame
# Filter for the year 2017
data_2017 = topic_map_unlabeled[topic_map_unlabeled['year'] == 2017]

# Separate NaN and non-NaN values
nan_values = data_2017[data_2017['Kk_pt_intensity'].isna()]
non_nan_values = data_2017[~data_2017['Kk_pt_intensity'].isna()]

# Cap the Kk_pt_intensity at 2
non_nan_values['Kk_pt_intensity_capped'] = non_nan_values['Kk_pt_intensity'].clip(upper=2)

# Set up the figure and axes for the histograms
industries = non_nan_values['ind12'].unique()
num_industries = len(industries)

# Create a figure for histograms
plt.figure(figsize=(15, 8))

# Create subplots for each industry
for i, industry in enumerate(industries):
    plt.subplot(np.ceil(num_industries / 4).astype(int), 4, i + 1)
    industry_data = non_nan_values[non_nan_values['ind12'] == industry]
    
    # Plot histogram for each industry
    plt.hist(industry_data['Kk_pt_intensity_capped'], bins=30, alpha=0.7, color='blue', edgecolor='black')
    plt.title(f'Kk_pt_intensity Distribution - Industry: {industry}')
    plt.xlabel('Kk_pt_intensity')
    plt.ylabel('Frequency')
    plt.xlim(0, 2)  # Set the x-limits from 0 to 2

plt.tight_layout()
plt.show()

# Calculate percentage of firms with NaN Kk_pt_intensity per industry
industry_nan_count = nan_values['ind12'].value_counts(normalize=True) * 100

# Create a count plot for the percentage of NaN values
plt.figure(figsize=(15, 5))
sns.barplot(x=industry_nan_count.index, y=industry_nan_count.values, palette='Blues')
plt.xticks(rotation=45)
plt.title('Percentage of NaN Kk_pt_intensity by Industry (2017)')
plt.xlabel('Industry (ind12)')
plt.ylabel('Percentage of Firms with NaN (%)')

plt.tight_layout()
plt.show()


KeyError: 'Kk_pt_intensity'

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
amazon_nov01_short['Date'] = pd.to_datetime(amazon_nov01_short['Date'])

# Find the specific 'nasdaq' and 'amazon' values for November 13, 2001
specific_date_values = amazon_nov01_short[amazon_nov01_short['Date'] == pd.to_datetime("2001-11-13")][['nasdaq', 'amazon']]

# Index 'nasdaq' and 'amazon' to 100 based on their values on November 13, 2001
amazon_nov01_short['nasdaq'] = 100 * amazon_nov01_short['nasdaq'] / specific_date_values['nasdaq'].values[0]
amazon_nov01_short['amazon'] = 100 * amazon_nov01_short['amazon'] / specific_date_values['amazon'].values[0]

# Plotting
plt.figure()
plt.plot('Date', 'nasdaq', data=amazon_nov01_short, color='blue', label='NASDAQ')
plt.plot('Date', 'amazon', data=amazon_nov01_short, color='red', label='Amazon')
plt.axvline(x=pd.to_datetime("2001-11-13"), linestyle='--', color='black')

# Setting the title, labels, legend, and formatting the x-axis dates
plt.title("NASDAQ vs Amazon Stock Prices in November 2001 (11/13/01 = 100)")
plt.xlabel("Date")
plt.ylabel("Stock Price")
plt.legend()
plt.xticks(rotation=45)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=1))
# Change font of the y-axis ticks:
plt.yticks(fontsize=8)
plt.grid(True)
plt.tight_layout()

# # Save the plot
plt.savefig(figfolder + "amazon_nov01.png", dpi = 600)
plt.close()